In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
import torch
import numpy as np

import os


/nfs/home/vshilonosov/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_lora_model(model_name, num_labels=2):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    lora_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=['lin1', 'lin2'],
        task_type="SEQ_CLS",
    )

    model = get_peft_model(model, lora_config)
    model = model.to("cuda")
    model.print_trainable_parameters()
    
    return model, tokenizer


def train_for_company(company, hf_dataset, model_name):
    model, tokenizer = create_lora_model(model_name)
    
    def tokenize(batch):
        return tokenizer(
            batch["message"], 
            padding="max_length", 
            truncation=True, 
            max_length=128
        )


    train_dataset = hf_dataset[company]["train"].map(tokenize, batched=True)
    test_dataset = hf_dataset[company]["test"].map(tokenize, batched=True)

    training_args = TrainingArguments(
        output_dir=f"./results/{company}",
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        learning_rate=3e-4,
        num_train_epochs=20,
        evaluation_strategy="steps",
        eval_steps=400,
        save_steps=400,
        logging_steps=400,
        logging_dir=f"./logs/{company}",
        save_strategy="epoch",
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits[0], axis=-1)
        accuracy = (predictions == labels).astype(np.float32).mean()
        return {"accuracy": accuracy}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    model.save_pretrained(f"./models/{company}")
    tokenizer.save_pretrained(f"./models/{company}")

    return model

In [3]:
df = pd.read_csv('final_data/df.csv')

/tmp/ipykernel_1646171/525276108.py:1: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_data/df.csv')


In [4]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True)

In [5]:
stocks = ['VTBR', 'GAZP', 'SBER', 'NVTK', 'ROSN']

hf_datasets = {}

for stock in stocks:
    stock_df = df[df[stock] == True].copy()
    stock_df = stock_df[['date', 'message', 'views', 'forwards', 'fwd_from', f'1 мин. {stock} close_bin']]
    stock_df.rename(columns={f'1 мин. {stock} close_bin': 'label'}, inplace=True)
    
    if len(stock_df) == 0:
        continue
    
    stock_df = stock_df.sort_values('date')
    
    split_idx = -1000
    train_data = stock_df.iloc[:split_idx]
    test_data = stock_df.iloc[split_idx:]

    hf_datasets[stock] = DatasetDict({
        'train': Dataset.from_pandas(train_data),
        'test': Dataset.from_pandas(test_data)
    })

In [6]:
dataset = DatasetDict(hf_datasets)

In [7]:
dataset.save_to_disk('final_data/hf_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 28391.30 examples/s]


In [9]:
dataset = load_from_disk('final_data/hf_dataset')

In [8]:
model_name = "DeepPavlov/distilrubert-base-cased-conversational"

In [9]:
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)

In [17]:
model = train_for_company('VTBR', dataset, model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,329,410 || all params: 136,655,620 || trainable%: 0.9728


Map: 100%|██████████| 1000/1000 [00:00<00:00, 7289.36 examples/s]
/nfs/home/vshilonosov/miniconda3/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
DistilBertSdpaAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Step,Training Loss,Validation Loss,Accuracy
400,0.661400,0.727608,0.526000
800,0.615800,0.780309,0.473000
1200,0.575400,0.793988,0.534000
1600,0.508300,0.945440,0.504000
2000,0.429200,1.031902,0.478000
2400,0.354500,1.261414,0.503000
2800,0.295000,1.424311,0.479000
3200,0.244200,1.503634,0.497000
3600,0.199900,1.718668,0.510000
4000,0.173700,1.727441,0.487000


In [10]:
model = train_for_company('GAZP', dataset, model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,329,410 || all params: 136,655,620 || trainable%: 0.9728


Map: 100%|██████████| 1000/1000 [00:00<00:00, 6472.18 examples/s]
/nfs/home/vshilonosov/miniconda3/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
DistilBertSdpaAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Step,Training Loss,Validation Loss,Accuracy
400,0.675400,0.723992,0.447000
800,0.640000,0.718170,0.420000
1200,0.624000,0.720203,0.440000
1600,0.614000,0.704411,0.527000
2000,0.596400,0.753997,0.444000
2400,0.587900,0.731755,0.493000
2800,0.573300,0.748886,0.477000
3200,0.539600,0.764351,0.528000
3600,0.528200,0.785062,0.513000
4000,0.480800,0.846829,0.534000


In [ ]:
model = train_for_company('SBER', dataset, model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,329,410 || all params: 136,655,620 || trainable%: 0.9728


Map: 100%|██████████| 1000/1000 [00:00<00:00, 5173.59 examples/s]
/nfs/home/vshilonosov/miniconda3/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
400,0.653000,0.720183,0.498000
800,0.607600,0.723890,0.508000
1200,0.574700,0.767928,0.499000
1600,0.524100,0.783316,0.493000
2000,0.463000,0.862782,0.501000
2400,0.401600,1.018550,0.510000
2800,0.350100,1.097298,0.487000


In [ ]:
model = train_for_company('NVTK', dataset, model_name)

In [ ]:
model = train_for_company('ROSN', dataset, model_name)